In [1]:
from transformers import pipeline
from tqdm import tqdm
import numpy as np
from itertools import product
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True,)

def emotion_scores_function(sample):
    emotion=classifier(sample)
    return emotion[0]


class BigramLM:
    def __init__(self):
        self.vocab = set()
        self.bigram_counts = {}
        self.unigram_counts = {}
        self.bigram_probs = {}
        self.emotions = ['joy','sadness','love','anger','fear','surprise']
        self.all_bigram_probs = {i:{} for i in self.emotions }
        self.total_prob = {i:0 for i in self.emotions   }

    def scale_bigram_probabilities(self, bigram_probabilities):

      # Find the score corresponding to the given emotion label

      total_probability = {}
      for bigram, prob in tqdm( bigram_probabilities.items(), desc="SCALING EMOTIONS"): #IMPLEMNET TQDM
          # Obtain emotional scores for the current bigram
          emotion_scores = emotion_scores_function(" ".join(bigram))

          for score in emotion_scores:
              scaled_probabilities = self.all_bigram_probs[score["label"]]
              scaled_probabilities[bigram] = prob * score["score"]
              self.total_prob[score["label"]]+=(prob * score["score"])

      for i in self.emotions:
        scaled_probabilities = self.all_bigram_probs[i]
        for bigram, prob in scaled_probabilities.items():
            scaled_probabilities[bigram] = prob / self.total_prob[score["label"]]




    def learn_model(self, corpus,smooth ="laplace"):
        # Implement code to learn bigram model from the dataset
        for sentence in corpus:
            tokens = sentence.split()
            # Add a start token at the beginning of each sentence
            tokens = ['<start>'] + tokens
            self.vocab.update(tokens)
        print(len(self.vocab))
        # Generate all possible bigrams using product
        all_bigrams = list(product(self.vocab, repeat=2))

        # Initialize bigram counts and unigram counts
        self.bigram_counts = {bigram: 0 for bigram in all_bigrams}
        self.unigram_counts = {word: 0 for word in self.vocab}

        # Update bigram and unigram counts
        for sentence in corpus:
            tokens = sentence.split()
            # Add a start token at the beginning of each sentence
            tokens = ['<start>'] + tokens
            for i in range(len(tokens) - 1):
                bigram = (tokens[i], tokens[i + 1])
                self.bigram_counts[bigram] += 1
                self.unigram_counts[tokens[i]] += 1

        time_modify_dict = {}
        count =500000
        for i,j in self.bigram_counts.items():
            if j !=0:
              time_modify_dict[i]=j
            else:
              if count:
                time_modify_dict[i]=j
                count-=1
        print(len(time_modify_dict),"kkook")
        self.bigram_counts = time_modify_dict
        # Calculate initial bigram probabilities
        if smooth ==None:
          for bigram, count in tqdm(self.bigram_counts.items(), desc="Generating probalities of bigram"):
              previous_word = bigram[0]
              if count == 0:
                  self.bigram_probs[bigram] = 0
              else:
                  self.bigram_probs[bigram] = count / self.unigram_counts[previous_word]
        elif smooth =="laplace":
          for bigram, count in tqdm(self.bigram_counts.items(), desc="Generating probalities of bigram"):
            previous_word = bigram[0]
            self.bigram_probs[bigram] = (count + 1) / (self.unigram_counts[previous_word] + len(self.vocab))
        elif smooth == "kneser-ney":
            bigram_counts = Counter(zip(corpus, corpus[1:]))
            c_star_1 = sum(1 for count in bigram_counts.values() if count == 1)
            c_star_2 = sum(1 for count in bigram_counts.values() if count == 2)
            d = c_star_1 / (c_star_1 + 2 * c_star_2)
            unigram_counts = Counter(corpus)
            total_bigrams = len(bigram_counts)

            for bigram in tqdm(self.bigram_counts.keys(), desc="Generating probabilities of bigram"):
                discounted_prob = max(bigram_counts.get(bigram, 0) - d, 0) / unigram_counts[bigram[1]]
                backoff_prob = sum(1 for key in bigram_counts.keys() if key[1] == bigram[1]) / total_bigrams
                self.bigram_probs[bigram] = discounted_prob + 0.5 * backoff_prob


        self.scale_bigram_probabilities(self.bigram_probs)


    def generate_next_word(self, current_word,emotion="fear"):

        if not self.bigram_probs:
            raise ValueError("Model has not been trained yet.")
        bigram_probs = self.all_bigram_probs[emotion]
        possible_next_words = [w2 for w1, w2 in bigram_probs if w1 == current_word and w2 !="<start>"]
        emotional_normlization = []

        probabilities = [bigram_probs.get((current_word, w2), 0) for w2 in tqdm(possible_next_words, desc="Choosing the next word")]
        probabilities = np.array(probabilities)

        # Normalize probabilities
        norm = probabilities / sum(probabilities)

        # Choose the next word based on probabilities
        next_word = np.random.choice(possible_next_words, p=norm)

        return next_word


# Example usage:
# Assuming you have a dataset, you can create an instance of BigramLM and train it on the dataset
# For simplicity, let's consider a small dataset:
file_path = 'D:\Downloads\corpus.txt'
corpus=[]
try:
    with open(file_path, 'r') as file:
        for line in file:
            corpus.append( line.strip())
except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")



bigram_model = BigramLM()
bigram_model.learn_model(corpus,None)

# Generate a sequence of words


C:\Users\Manvendra Nema\anaconda3\envs\general\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Manvendra Nema\anaconda3\envs\general\Lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


5430
524452 kkook


SCALING EMOTIONS: 100%|██████████████████████████████████████████████████████| 524452/524452 [1:46:29<00:00, 82.08it/s]


In [2]:
# import pickle

# with open('no_vocab.pkl', 'wb') as file:
#     pickle.dump(bigram_model.vocab, file)

# with open('no_bigram_counts.pkl', 'wb') as file:
#     pickle.dump(bigram_model.bigram_counts, file)

# with open('no_unigram_counts.pkl', 'wb') as file:
#     pickle.dump(bigram_model.unigram_counts, file)

# with open('no_bigram_probs.pkl', 'wb') as file:
#     pickle.dump(bigram_model.bigram_probs, file)

# with open('no_model.pkl', 'wb') as file:
#     pickle.dump(bigram_model, file)

# with open('no_total_prob.pkl', 'wb') as file:
#     pickle.dump(bigram_model.total_prob, file)

# with open('no_all_bigram_probs.pkl', 'wb') as file:
#     pickle.dump(bigram_model.all_bigram_probs, file)


In [4]:
import pickle
from transformers import pipeline
from tqdm import tqdm
import numpy as np
from itertools import product
# Load vocabulary
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_vocab.pkl', 'rb') as file:
    loaded_vocab = pickle.load(file)

# Load bigram counts
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_bigram_counts.pkl', 'rb') as file:
    loaded_bigram_counts = pickle.load(file)

# Load unigram counts
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_unigram_counts.pkl', 'rb') as file:
    loaded_unigram_counts = pickle.load(file)

# Load bigram probabilities
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_bigram_probs.pkl', 'rb') as file:
    loaded_bigram_probs = pickle.load(file)

# Load the entire bigram model
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_model.pkl', 'rb') as file:
    bigram_model = pickle.load(file)

# Load total probability
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_total_prob.pkl', 'rb') as file:
    loaded_total_prob = pickle.load(file)

# Load all bigram probabilities
with open(r'C:\Users\Manvendra Nema\Notebooks\Submission_NLP_A1\no_all_bigram_probs.pkl', 'rb') as file:
    loaded_all_bigram_probs = pickle.load(file)


In [22]:
z = []
for i,j in loaded_bigram_probs.items():
    if loaded_bigram_counts[i] and i[0]=='i' and i[1] in ['wanna', 'gain', 'guess', 'once', 'meant']:
        z.append((i,j))
for i in sorted(z,key = lambda x: x[1]):
    print(i)

(('i', 'wanna'), 0.0002642706131078224)
(('i', 'meant'), 0.0002642706131078224)
(('i', 'gain'), 0.0002642706131078224)
(('i', 'once'), 0.0002642706131078224)
(('i', 'guess'), 0.003171247357293869)


In [8]:
z = []

for em,tu in loaded_all_bigram_probs.items():
    count =15
    print(em)
    for a in tu:
        if count==0:
            break
        if a[0]=='i':
            print(a,": ", loaded_all_bigram_probs[em][a])
            count-=1
        
    # print(loaded_all_bigram_probs['joy'][('i','am')])

joy
('i', 'hide') :  8.02807862861471e-07
('i', 'remembered') :  2.526979445611897e-06
('i', 'can') :  0.0002920104923740879
('i', 'simply') :  4.292072931131094e-06
('i', 'dunno') :  3.7975560545506886e-06
('i', 'also') :  5.736764028794021e-05
('i', 'looked') :  9.915085179128714e-07
('i', 'hardly') :  8.393713438139714e-08
('i', 'cringe') :  3.5154019861593887e-07
('i', 'imagine') :  2.8128306467843843e-06
('i', 'a') :  6.589350714763712e-07
('i', 'confess') :  1.1178106339916291e-06
('i', 'bore') :  8.17317498917669e-07
('i', 'work') :  8.402090986131024e-06
('i', 'log') :  3.6773470178336305e-06
sadness
('i', 'hide') :  2.6394409649324174e-07
('i', 'remembered') :  1.1095445639512247e-06
('i', 'can') :  8.12811307329125e-06
('i', 'simply') :  4.02123847915627e-07
('i', 'dunno') :  1.9345331071918766e-07
('i', 'also') :  4.408354171800414e-06
('i', 'looked') :  6.358052126847919e-07
('i', 'hardly') :  8.976290117072668e-08
('i', 'cringe') :  1.761780605623681e-07
('i', 'imagine') :

In [10]:
def generate_word_sequences(bigram_model, emotion, num_sequences=1, max_length=50):
    all_sequences = []

    for _ in range(num_sequences):
        current_word = "<start>"
        generated_sequence = []

        for _ in range(max_length):
            try:
                if current_word!= "<start>":
                    generated_sequence.append(current_word)
                current_word = bigram_model.generate_next_word(current_word, emotion)

            except Exception as e:
                current_word = bigram_model.generate_next_word("<start>", emotion)

        all_sequences.append(' '.join(generated_sequence)[1:])

    return all_sequences

# Example usage:
emotion_input = 'surprise'
num_sequences = 1
generated_sequences = generate_word_sequences(bigram_model, emotion_input, num_sequences,20)
for i, sequence in enumerate(generated_sequences, start=1):
    print(f"Sequence {i}: {sequence}")


Choosing the next word: 100%|██████████████████████████████████████████████████████████████████| 33/33 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|██████████████████████████████████████████████████████████████████| 20/20 [00:00<?, ?it/s]

Sequence 1:  may mention a dorky unfit rotund sloth that is too weird prose i discover his family events which


In [11]:
def generate_word_sequences(bigram_model, emotion, num_sequences=1, max_length=50):
    all_sequences = []

    for _ in range(num_sequences):
        current_word = "<start>"
        generated_sequence = []

        for _ in range(max_length):
            try:
                if current_word != "<start>":
                    generated_sequence.append(current_word)
                current_word = bigram_model.generate_next_word(current_word, emotion)

            except Exception as e:
                current_word = bigram_model.generate_next_word("<start>", emotion)

        all_sequences.append(' '.join(generated_sequence)[1:])

    return all_sequences


# Example usage:
emotions = ['joy', 'sadness', 'love', 'anger', 'fear', 'surprise']
num_sequences_per_emotion = 50
generated_corpus = []
generated_labels = []

for emotion in emotions:
    generated_sequences = generate_word_sequences(bigram_model, emotion, num_sequences_per_emotion, 25)
    generated_corpus.extend(generated_sequences)
    generated_labels.extend([emotion] * num_sequences_per_emotion)

# Print or use the generated corpus and labels as needed
for i, (sequence, label) in enumerate(zip(generated_corpus, generated_labels), start=1):
    print(f"Sequence {i}: {sequence} - Emotion: {label}")


Choosing the next word: 100%|████████████████████████████████████████████████████████████████| 109/109 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████| 673/673 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|██████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 3996.48it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word

Sequence 1:  feel like buy to happen to feel a normal as well lived is supporting palin for a feeling of the possibility of how - Emotion: joy
Sequence 2:  feel like there have been going to be in the st century college with the feeling so feel the community and enjoy it - Emotion: joy
Sequence 3:  like i feel the glories of ideas how transparent i feel like i learned over and its really hard time passing sentence in - Emotion: joy
Sequence 4:  feel the same time for staying home feeling terrific stuff so contented after spending time especially passionate about living makes me feel worthwhile - Emotion: joy
Sequence 5:  am here that did or feel really really console port of being quite alright nothing else sees but feeling from well thought action - Emotion: joy
Sequence 6:  will be there is just find when for those childhood favorite activities as well as evidenced in a good old custom orthotics and - Emotion: joy
Sequence 7:  was popular distribution releases his words just find a f

In [43]:
import os
# Example usage:
emotions = ['joy', 'sadness', 'love', 'anger', 'fear', 'surprise']
# Create a directory to store generated samples
output_directory = "generated_samples_n"
os.makedirs(output_directory, exist_ok=True)

def generate_and_save_sequences(bigram_model, emotion, num_sequences=50, max_length=20):
    generated_sequences = generate_word_sequences(bigram_model, emotion, num_sequences, max_length)

    # Save the generated sequences to a file
    output_filename = f"{output_directory}/gen_{emotion.lower()}.txt"
    with open(output_filename, 'w') as file:
        for sequence in generated_sequences:
            file.write(f"{sequence}\n")

    return output_filename

# Example usage:
num_sequences_per_emotion = 50
for emotion in emotions:
    generated_file = generate_and_save_sequences(bigram_model, emotion, num_sequences_per_emotion, 7)
    print(f"Generated {num_sequences_per_emotion} samples for {emotion}: {generated_file}")


Choosing the next word: 100%|████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████| 324/324 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████| 673/673 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████████████████████| 7/7 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|██████████████████████████████████████████████████████████████████| 16/16 [00:00<?, ?it/s]


Generated 50 samples for joy: generated_samples_n/gen_joy.txt


Choosing the next word: 100%|█████████████████████████████████████████████████| 5429/5429 [00:00<00:00, 1428357.57it/s]
C:\Users\Manvendra Nema\AppData\Local\Temp\ipykernel_11120\3447452138.py:121: RuntimeWarning: divide by zero encountered in divide
  norm = probabilities / sum(probabilities)
C:\Users\Manvendra Nema\AppData\Local\Temp\ipykernel_11120\3447452138.py:121: RuntimeWarning: invalid value encountered in divide
  norm = probabilities / sum(probabilities)
Choosing the next word: 100%|██████████████████████████████████████████████████████████████████| 59/59 [00:00<?, ?it/s]


Generated 50 samples for sadness: generated_samples_n/gen_sadness.txt


Choosing the next word: 100%|██████████████████████████████████████████████████████████████████| 13/13 [00:00<?, ?it/s]
Choosing the next word: 0it [00:00, ?it/s]
Choosing the next word: 100%|████████████████████████████████████████████████████| 397/397 [00:00<00:00, 395801.92it/s]


...

Generated 50 samples for anger: generated_samples_n/gen_anger.txt


Choosing the next word: 100%|████████████████████████████████████████████████████| 433/433 [00:00<00:00, 433755.35it/s]


Generated 50 samples for fear: generated_samples_n/gen_fear.txt


Choosing the next word: 100%|████████████████████████████████████████████████████████████████| 673/673 [00:00<?, ?it/s]

Generated 50 samples for surprise: generated_samples_n/gen_surprise.txt


In [44]:
emotions = ['joy', 'sadness', 'love', 'anger', 'fear', 'surprise']
def read_generated_sequences_and_labels(emotion):
    input_filename = f"{output_directory}/gen_{emotion.lower()}.txt"
    generated_sequences = []
    labels = []  # Assign the emotion label to all sequences

    try:
        with open(input_filename, 'r') as file:
            for line in file:
                generated_sequences.append(line.strip())
                labels.append(emotion)
    except FileNotFoundError:
        print(f"File '{input_filename}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return generated_sequences, labels

# Example usage:
all_generated_sequences = []
all_generated_labels = []

for emotion in emotions:
    generated_sequences, labels = read_generated_sequences_and_labels(emotion)
    all_generated_sequences.extend(generated_sequences)
    all_generated_labels.extend(labels)


300

In [23]:
# with open(r"D:\Downloads\tfidf_svc_model.pkl", 'rb') as file:
#     loaded_label_encoder, loaded_tfidf_vectorizer, loaded_svc = pickle.load(file)

In [45]:
from sklearn.metrics import accuracy_score, f1_score, classification_report


# Assuming you have loaded the components using pickle
# with open('model.pkl', 'rb') as file:
#     loaded_label_encoder, loaded_tfidf_vectorizer, loaded_svc, loaded_grid_search = pickle.load(file)

# Sample testing data
test_corpus = all_generated_sequences
test_labels = all_generated_labels

# Transform labels using the loaded label encoder
encoded_test_labels = loaded_label_encoder.transform(test_labels)

# Use the loaded TF-IDF vectorizer and SVC model for prediction
X_test = loaded_tfidf_vectorizer.transform(test_corpus)
y_pred = loaded_svc.predict(X_test)
# print(y_pred)
# Decode the predicted labels back to original labels
# decoded_pred_labels = loaded_label_encoder.inverse_transform(y_pred)
# print(decoded_pred_labels)
# Calculate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(test_labels, y_pred, average='macro')
print("F1 Score:", f1)

# Generate classification report
report = classification_report(test_labels, y_pred, target_names=loaded_label_encoder.classes_)
print("Classification Report:")
print(report)

Accuracy: 0.5433333333333333
F1 Score: 0.5386189472906003
Classification Report:
              precision    recall  f1-score   support

       anger       0.28      0.20      0.23        50
        fear       0.55      0.42      0.48        50
         joy       0.35      0.44      0.39        50
        love       0.75      0.86      0.80        50
     sadness       0.49      0.68      0.57        50
    surprise       0.89      0.66      0.76        50

    accuracy                           0.54       300
   macro avg       0.55      0.54      0.54       300
weighted avg       0.55      0.54      0.54       300

